Author: Matthew Viafora

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imports:

In [429]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc

Load datasets into Pandas dataframes:

In [430]:
t_train = pd.read_csv("/kaggle/input/titanic/train.csv")
t_test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [431]:
t_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Remove survived column from t_train and store in y variable:

Remove Name column as well (not needed):

In [432]:
y_train = t_train.pop("Survived")
t_train.pop("Name")
t_test.pop("Name")
t_train.pop("Ticket")
t_test.pop("Ticket")
t_train.pop("Cabin")
t_test.pop("Cabin")
pass_id = t_test.pop("PassengerId")
t_train.pop("PassengerId")

0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int64

In [433]:
t_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


Fill "nan" cells in "Embark" and "Sex" columns:

In [434]:
# for i in t_train["Embarked"]:
#     if pd.isnull(i) == True:
#         t_train.at[i, 'Embarked'] = "N"
t_train['Embarked'] = t_train['Embarked'].fillna("N")
t_train['Sex'] = t_train['Sex'].fillna("N")

Age has 263 missing values, I will replace them with the mean age of all other ages in category in order to not void the entire column completely:

In [435]:
age_avg = t_train['Age'].mean()
t_train['Age'].fillna(t_train['Age'].median(), inplace = True)
t_test['Age'].fillna(t_test['Age'].median(), inplace = True)
t_train['Embarked'].fillna(t_train['Embarked'].mode()[0], inplace = True)
t_test['Embarked'].fillna(t_test['Embarked'].mode()[0], inplace = True)
t_train['Fare'].fillna(t_train['Fare'].median(), inplace = True)
t_test['Fare'].fillna(t_test['Fare'].median(), inplace = True)

Feature columns:

In [436]:
CATEGORICAL_COLUMNS = ["Sex", "Embarked"]
NUMERICAL_COLUMNS = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
feature_columns = []

Get unique vocabulary from all categorical columns and place into appropriate feature column:

In [437]:
for i in CATEGORICAL_COLUMNS:
    vocabulary = t_train[i].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(i, vocabulary))

Do the same for numeric columns:

In [438]:
for i in NUMERICAL_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(i, dtype=tf.float32))

Make input function for tf linear classifier:

In [439]:
def make_input_fn(data_df, label_df, num_epochs=10,shuffle=True,batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

def make_input_fn_test(data_df, num_epochs=10,shuffle=True,batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df)))
        if shuffle:
            ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

train_input_fn = make_input_fn(t_train, y_train)
test_input_fn = make_input_fn_test(t_test, num_epochs=1, shuffle=False)

Create linear estimator:

In [440]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

Print out feature columns for reference:

In [441]:
for i in feature_columns:
    print(i)

VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
VocabularyListCategoricalColumn(key='Embarked', vocabulary_list=('S', 'C', 'Q', 'N'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
NumericColumn(key='Pclass', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


Train model:

In [442]:
t_train.info()
print("______________________________________________________________________________________________________________")
t_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB
______________________________________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    in

In [443]:
linear_est.train(train_input_fn)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [444]:
result = linear_est.predict(test_input_fn)
# for i in result:
#         print('result: {}'.format(i))
#         print("\n")
outputs = []
while result:
    try:
        it = next(result)['probabilities']
        if it[0] > it[1]:
            outputs.append(0)
        else:
            outputs.append(1)
    except StopIteration:
        break
#next(result)['probabilities'][1]

submission = pd.DataFrame({
       "PassengerId": pass_id,
        "Survived": outputs
    })
submission.to_csv('viafora1_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
